# Parkinglot Finder Walkthrough
This is a jupyter notebook walkthrough of the pipeline from a tif file to outputing a GeoJson of the parkinlots identified in the image. Make sure to follow the [installation instructions](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/installation.md) before you start.

# Imports

In [192]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import pickle
from osgeo import gdal

from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image

# This is needed since the notebook is stored in the object_detection folder.
sys.path.append("..")
from object_detection.utils import ops as utils_ops

if tf.__version__ < '1.4.0':
  raise ImportError('Please upgrade your tensorflow installation to v1.4.* or later!')


# File Input
Place the file name in the quotations below. Note that the file must be a TIF file (meaning that the extension must be .tif).

In [193]:
file_name =  "Vegas_333.tif"

if file_name.__len__ () == 0 or file_name.rfind (".tif") == -1:
    raise EnvironmentError ("Input file must be a TIF image, got: " + file_name)

## Env setup

In [194]:
# This is needed to display the images.
%matplotlib inline

## Object detection imports
Here are the imports from the object detection module.

In [195]:
from utils import label_map_util

from utils import visualization_utils as vis_util

# Model preparation 

## Variables

Here `MODEL_NAME` should be changed to the name of the inference graph. Additionally, you should change `PATH_TO_LABELS` to the appropriate object-detection.pbtxt file and adjust `NUM_CLASSES` to represent the number of classes your model outputs.

In [196]:
# What model to download.
MODEL_NAME = 'parking_lot_inference_graph'

# Path to frozen detection graph. This is the actual model that is used for the object detection.
PATH_TO_CKPT = MODEL_NAME + '/frozen_inference_graph.pb'

# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = os.path.join('/home/thomas/Documents/SatImageParkingLotFinder/training', 'object-detection.pbtxt')

NUM_CLASSES = 1

## Load a (frozen) Tensorflow model into memory.

In [197]:
detection_graph = tf.Graph()
with detection_graph.as_default():
  od_graph_def = tf.GraphDef()
  with tf.gfile.GFile(PATH_TO_CKPT, 'rb') as fid:
    serialized_graph = fid.read()
    od_graph_def.ParseFromString(serialized_graph)
    tf.import_graph_def(od_graph_def, name='')

## Loading label map
This creates the label map that maps the numeric output to a specific label so since we have only once class it will map `1` to `parking lot`

In [198]:
label_map = label_map_util.load_labelmap(PATH_TO_LABELS)
categories = label_map_util.convert_label_map_to_categories(label_map, max_num_classes=NUM_CLASSES, use_display_name=True)
category_index = label_map_util.create_category_index(categories)

## Helper code

In [199]:
def load_image_into_numpy_array(image):
  (im_width, im_height) = image.size
  return np.array(image.getdata()).reshape(
      (im_height, im_width, 3)).astype(np.uint8)

# Detection

In [200]:
# For the sake of simplicity we will use only 2 images:
# image1.jpg
# image2.jpg
# If you want to test the code with your images, just add path to the images to the TEST_IMAGE_PATHS.
PATH_TO_TEST_IMAGES_DIR = ''
image_path = os.path.join(PATH_TO_TEST_IMAGES_DIR, file_name.replace('tif','jpeg'))
# Size, in inches, of the output images.
IMAGE_SIZE = (24, 16)

In [201]:
def run_inference_for_single_image(image, graph):
  with graph.as_default():
    with tf.Session() as sess:
      # Get handles to input and output tensors
      ops = tf.get_default_graph().get_operations()
      all_tensor_names = {output.name for op in ops for output in op.outputs}
      tensor_dict = {}
      for key in [
          'num_detections', 'detection_boxes', 'detection_scores',
          'detection_classes', 'detection_masks'
      ]:
        tensor_name = key + ':0'
        if tensor_name in all_tensor_names:
          tensor_dict[key] = tf.get_default_graph().get_tensor_by_name(
              tensor_name)
      if 'detection_masks' in tensor_dict:
        # The following processing is only for single image
        detection_boxes = tf.squeeze(tensor_dict['detection_boxes'], [0])
        detection_masks = tf.squeeze(tensor_dict['detection_masks'], [0])
        # Reframe is required to translate mask from box coordinates to image coordinates and fit the image size.
        real_num_detection = tf.cast(tensor_dict['num_detections'][0], tf.int32)
        detection_boxes = tf.slice(detection_boxes, [0, 0], [real_num_detection, -1])
        detection_masks = tf.slice(detection_masks, [0, 0, 0], [real_num_detection, -1, -1])
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
            detection_masks, detection_boxes, image.shape[0], image.shape[1])
        detection_masks_reframed = tf.cast(
            tf.greater(detection_masks_reframed, 0.5), tf.uint8)
        # Follow the convention by adding back the batch dimension
        tensor_dict['detection_masks'] = tf.expand_dims(
            detection_masks_reframed, 0)
      image_tensor = tf.get_default_graph().get_tensor_by_name('image_tensor:0')

      # Run inference
      output_dict = sess.run(tensor_dict,
                             feed_dict={image_tensor: np.expand_dims(image, 0)})

      # all outputs are float32 numpy arrays, so convert types as appropriate
      output_dict['num_detections'] = int(output_dict['num_detections'][0])
      output_dict['detection_classes'] = output_dict[
          'detection_classes'][0].astype(np.uint8)
      output_dict['detection_boxes'] = output_dict['detection_boxes'][0]
      output_dict['detection_scores'] = output_dict['detection_scores'][0]
      if 'detection_masks' in output_dict:
        output_dict['detection_masks'] = output_dict['detection_masks'][0]
  return output_dict


In [202]:
from __future__ import absolute_import
import os

GDAL_COMMAND    = u"gdal_translate -scale_1 20 1463 -scale_2 114 1808 -scale_3 139 1256 -ot Byte -of"
file_type       = None

file_types = ["jpeg", "png"]

def convertTifTo (file_type, file_name):
    u'''
        Converts the given TIF file to the given file format.

        @param  file_type   - The file format to convert to \n
        @param  file_name   - The TIF file to convert
    '''
    type_str = file_type.__str__ ()
    conv_str = file_name.replace (u"tif", type_str)

    os.system (GDAL_COMMAND + u" " + type_str.upper () + u" " + file_name + u" " + conv_str)
    os.system (u"rm " + conv_str + u".aux.xml")

def generateArguments ():
    u'''
        Parses the arguments passed to the script, and sets fields appropriately.

        @throws EnvironmentError if an argument couldn't be understood (i.e. file format was not understood).
    '''

    from sys import argv
    
    args = {}
    global file_name, file_type

    while argv:
        if argv [0][0] == u"-":
            args [argv [0]] = argv [1]

        argv = argv [1:]

    if u"-t" in args:
        arg = args [u"-t"]

        for t in file_types:
            if arg in t:
                file_type = t
                break

        if file_type is None:
            raise EnvironmentError (u"Conversion file format was not recognized; Use: " + File_Type.list ())
    
    else:
        raise EnvironmentError (u"A file format to convert to must be specified: -t " + File_Type.list ())

    if u"-f" in args:
        file_name = args [u"-f"]

def walkThrough (root = None):
    u'''
        Walks through the given directory and subdirectories, and converts any TIF files found to
        the file format given as an argument to the script.

        @param  root - The directory to begin the conversion at
    '''

    if root is None:
        root = u"."

    for dirpath, dirnames, filenames in os.walk (root):
        for file in filenames:
            if file.endswith (u".tif"):
                convertTifTo (file_type, os.path.join (dirpath, file))
            
convertTifTo (file_types [0], file_name)

In [203]:
from __future__ import absolute_import
from sys import argv
import json

# http://lxml.de/
from lxml import etree

# http://www.gdal.org/index.html
from osgeo import gdal
import io

geojson = {
    u"type" : u"FeatureCollection",
    u"name" : file_name,
    u"features" : []
}

def appendFeature (geojson, rect):
    u'''
        Appends a feature representing the given bounding box to the given
        dictionary.

        @param  geojson - The dictionary to append the new feature to \n
        @param  rect    - The list representing the bounding box (as x1, y1, x2, y2)
    '''
    geojson [u"features"].append ({
        u"type": u"Feature",
        u"properties": {},
        u"geometry": {
            u"type": u"Polygon",
            u"coordinates": [ rect ]
        }
    })

def convertCoordinates (dataset, x, y):
    u'''
        Converts the given pixel coordinates to geographical coordinates using the given
        GDAL data set.

        @param  dataset - The GDAL data set to extract latitude / longitude from \n
        @param  x       - The x pixel coordinate \n
        @param  y       - The y pixel coordinate \n

        @return A tuple containing the pixel coordinates converted to geographical coordinates
    '''
    origin = getOrigin (dataset)
    pixel_size = getPixelSize (dataset)

    return x * pixel_size [0] + origin [0], y * pixel_size [1] + origin [1]

def generateArguments ():    
    u'''
        Parses the program arguments to determine what file should be converted.

        @throws EnvironmentError if the argument couldn't be found / parsed
    '''
    global argv, file_name
    args = {}

    while argv:
        if argv [0][0] == u"-":
            args [argv [0]] = argv [1]

        argv = argv [1:]

    if u"-f" in args:
        arg = args [u"-f"]
        n = arg.rfind (u".")

        file_name = arg [:n] if n >= 0 else arg

    else:
        raise EnvironmentError (u"Filename must be specified with -f")

def getCoordinates (dataset, bndbox):
    u'''
        Gets and converts the coordinates of the bounding box contained in the given tree
        element.

        @param  dataset     - The GDAL data set to extract latitude / longitude from \n
        @param  robndbox    - The XML tree element that contains the bounding box (robndbox tag)

        @return A list containing the converted coordinates of the bounding box
    '''
    width, height = getImageSize (dataset)
    
    xmin = float (bndbox [1]) * width
    ymin = float (bndbox [0]) * height
    xmax = float (bndbox [3]) * width
    ymax = float (bndbox [2]) * height

    return [convertCoordinates (dataset, xmin, ymin),
        convertCoordinates (dataset, xmin, ymax),
        convertCoordinates (dataset, xmax, ymax),
        convertCoordinates (dataset, xmax, ymin),
        convertCoordinates (dataset, xmin, ymin)]

def getImageSize (dataset):
    return dataset.RasterXSize, dataset.RasterYSize

def getOrigin (dataset):
    u'''
        Returns the origin of the given GDAL data set.

        @param  dataset - The GDAL data set to get the origin (geographical coordinates) of

        @return A tuple containing the x and y geographical coordinates of the origin
    '''
    geotransform = dataset.GetGeoTransform ()
    return geotransform [0], geotransform [3]

def getPixelSize (dataset):
    u'''
        Returns the pixel size of each pixel in the given GDAL data set.

        @param  dataset - The GDAL data set to get the pixel size of

        @return A tuple containing the x and y pixel sizes
    '''
    geotransform = dataset.GetGeoTransform ()
    return geotransform [1], geotransform [5]

def convertBoundingBox (box, dataset):
    appendFeature (geojson, getCoordinates (dataset, box))
    

In [205]:

image = Image.open(image_path)
# the array based representation of the image will be used later in order to prepare the
# result image with boxes and labels on it.
image_np = load_image_into_numpy_array(image)
# Expand dimensions since the model expects images to have shape: [1, None, None, 3]
image_np_expanded = np.expand_dims(image_np, axis=0)
# Actual detection.

data = gdal.Open (file_name, gdal.GA_ReadOnly)
output_dict = run_inference_for_single_image(image_np, detection_graph)
for i in range(0, output_dict['detection_boxes'].shape[0]):
    #print(output_dict['detection_scores'][i]
    if output_dict['detection_scores'][i] > 0.8:
            #print(output_dict['detection_scores'][i])
            convertBoundingBox (output_dict['detection_boxes'][i], data)
              

with io.open(file_name.replace ('tif', 'geojson'),'w',encoding="utf-8") as outfile:
    outfile.write(unicode(json.dumps(geojson, ensure_ascii=False)))
   # Visualization of the results of a detection.